In [1]:
from pprint import pprint

import parser
import ast
import inspect

import numpy as np
import pandas as pd

In [2]:
def add(x, y=0):
    return

In [3]:
# Label
blha = add(
    1,
    y=2+3,
)

In [4]:
text = In[3]
print(text)

# Label
blha = add(
    1,
    y=2+3,
)


In [5]:
lines = text.split('\n')
print(lines)

comment_lines = [l.strip() for l in lines if l.startswith('#')]
print(comment_lines)

label = ''.join(comment_lines[0].split('#')[1:]).strip()
print(label)

['# Label', 'blha = add(', '    1,', '    y=2+3,', ')']
['# Label']
Label


'add'

In [6]:
returns = []
m = ast.parse(text)
b = m.body[0]
if isinstance(b, ast.Assign):
    for t in b.targets:
        returns.append({
            'label': 'TODO: get from docstring',
            'description': 'TODO: get from docstring',
            'value': t.id,
        })
pprint(returns)

[{'description': 'TODO: get from docstring',
  'label': 'TODO: get from docstring',
  'value': 'blha'}]


In [7]:
code_lines = [l.strip() for l in lines if not l.startswith('#')]
print(code_lines)

['blha = add(', '1,', 'y=2+3,', ')']


In [13]:
function_name = b.value.func.id
print(function_name)

add


In [14]:
args = []
kwargs = {}

for a in [l for l in code_lines if not (l.endswith('(') or l.startswith(')'))]:
    
    if '=' in a:  # kwarg
        k, v = a.split('=')
        if v.endswith(','):
            v = v[:-1]
        kwargs[k] = v
        
    else:  # arg
        if a.endswith(','):
            a = a[:-1]
        args.append(a)
        
print(args)
print(kwargs)

['1']
{'y': '2+3'}


In [15]:
s = inspect.signature(add)

for k, v in s.parameters.items():
    print(k, v)

x x
y y=0


In [16]:
from pprint import pprint

# Get required args
required_args = [{
    'label': n,
    'description': 'Description.',
    'name': n,
    'value': v,
} for n, v in zip([v.name for v in s.parameters.values() if v.default == inspect._empty], args)]
pprint(required_args)

# Get optional args
optional_args = [{
    'label': n,
    'description': 'Description.',
    'name': n,
    'value': v,
} for n, v in kwargs.items()]
pprint(optional_args)

[{'description': 'Description.', 'label': 'x', 'name': 'x', 'value': '1'}]
[{'description': 'Description.', 'label': 'y', 'name': 'y', 'value': '2+3'}]


In [18]:
# Get module name
library_name = eval('{}.__module__'.format(function_name))
print(library_name)

# Get module path
if library_name == '__main__':  # Function is defined within this Notebook
    library_path = ''
else:  # Function is imported from a module)
    library_path = eval(
        '{}.__globals__.get(\'__file__\')'.format(function_name)
    ).split(library_name.replace('.', '/'))[0]
print(library_path)

# Make a task
task = {
    label: {
        'description': 'Description.',
        'library_path': library_path,
        'library_name': library_name,
        'function_name': function_name,
        'required_args': required_args,
        'default_args': {},
        'optional_args': optional_args,
        'returns': returns
    }
}

# Register this task
self._update_tasks(task)

return task

__main__

